In [ ]:
!pip install pandarallel
from pandarallel import pandarallel

import pandas as pd
import csv

import numpy as np
from sklearn.model_selection import StratifiedKFold

from google.colab import drive
drive.mount('/content/drive')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import text
from sklearn.pipeline import FeatureUnion,Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn_pandas import DataFrameMapper
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import pickle


%cd /content/drive/My\ Drive/SNA2020/Project_kremlinbots

def metrics(clf, X, y):
  scores = cross_validate(clf, X, y, cv=5, scoring=['accuracy','f1_macro'])
  res = [scores['test_accuracy'].mean(), scores['test_f1_macro'].mean()]
  print(res)

def learning(X, y, mapper, min_max_scaler, name):
  print("LinearSVC()")
  metrics(LinearSVC(), X, y)
  print("RandomForestClassifier()")
  metrics(RandomForestClassifier(n_estimators=300), X, y)
  model = RandomForestClassifier(n_estimators=300)
  model.fit(X, y)
  pkl_filename = name + "_RandomForestClassifier_model.pkl"
  tuple_objects = (model, mapper, min_max_scaler)
  with open(pkl_filename, 'wb') as file:
    pickle.dump(tuple_objects, file)
  print("GaussianNB()")
  metrics(GaussianNB(), X, y)
  print("AdaBoostClassifier()")
  metrics(AdaBoostClassifier(), X, y)
  print("LogisticRegression()")
  metrics(LogisticRegression(), X, y)

def processing(df_all, name):
  print(name)
  mapper = DataFrameMapper([
     (name, text.TfidfVectorizer(max_features=1400)),
     ('has_media', None),
     ('time_dif', None),
     ('likes_cnt', None),
     ('self_like', None),
     ('replies_cnt', None),
     ('reg_date_dif', None),
     ('is_closed', None),
  ])
  y = df_all['is_bot']
  X = mapper.fit_transform(df_all)
  min_max_scaler = preprocessing.MinMaxScaler()
  np_scaled = min_max_scaler.fit_transform(X)
  X = pd.DataFrame(np_scaled) 
  learning(X, y, mapper, min_max_scaler, name)
  print(X)
  print("_______________________________________")

df_all_pymorphy_nltk_stopwords_badwords=pd.read_csv('./all_pymorphy_nltk-stopwords_badwords.tsv',sep="\t",header=0)
processing(df_all_pymorphy_nltk_stopwords_badwords, "pymorphy_nltk-stopwords_badwords")
df_all_pymorphy_github_stopwords_badwords=pd.read_csv('./all_pymorphy_github-stopwords_badwords.tsv',sep="\t",header=0)
processing(df_all_pymorphy_github_stopwords_badwords, "pymorphy_github-stopwords_badwords")
df_all_pymorphy_nltk_stopwords_github_stopwords_badwords=pd.read_csv('./all_pymorphy_nltk-stopwords_github-stopwords_badwords.tsv',sep="\t",header=0)
processing(df_all_pymorphy_nltk_stopwords_github_stopwords_badwords, "pymorphy_nltk-stopwords_github-stopwords_badwords")

df_all_mystem_nltk_stopwords_badwords=pd.read_csv('./all_mystem_nltk-stopwords_badwords.tsv',sep="\t",header=0)
processing(df_all_mystem_nltk_stopwords_badwords, "mystem_nltk-stopwords_badwords")
df_all_mystem_github_stopwords_badwords=pd.read_csv('./all_mystem_github-stopwords_badwords.tsv',sep="\t",header=0)
processing(df_all_mystem_github_stopwords_badwords, "mystem_github-stopwords_badwords")
df_all_mystem_nltk_stopwords_github_stopwords_badwords=pd.read_csv('./all_mystem_nltk-stopwords_github-stopwords_badwords.tsv',sep="\t",header=0)
processing(df_all_mystem_nltk_stopwords_github_stopwords_badwords, "mystem_nltk-stopwords_github-stopwords_badwords")